# Lecture de la matrice DOPE

In [5]:
import pandas as pd
import numpy as np

In [2]:
def clean_DOPE_data(filename):
    ca_matrix = []
    
    with open(filename, "r") as dope :
        for ligne in dope:
            if ligne[3:7].strip() == "CA" and ligne[11:14].strip() == "CA":
                ca_matrix.append(ligne.split())
    
    columns = ['res1', 'temp1', 'res2', 'temp2'] + list(np.arange(0.25, 15, 0.5))
    dope_score = pd.DataFrame(ca_matrix, columns = columns) 
    dope_score = dope_score.drop(['temp1', 'temp2'], axis=1)
    
    return dope_score

In [3]:
DOPE_FILE = "dope.par.txt"
DOPE_MATRIX = clean_DOPE_data(DOPE_FILE)
print(DOPE_MATRIX)

    res1 res2   0.25   0.75   1.25   1.75   2.25   2.75   3.25   3.75  ...  \
0    ALA  ALA  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.64  ...   
1    ALA  ARG  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.61  ...   
2    ALA  ASN  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.58  ...   
3    ALA  ASP  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.62  ...   
4    ALA  CYS  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.49  ...   
..   ...  ...    ...    ...    ...    ...    ...    ...    ...    ...  ...   
395  VAL  SER  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.48  ...   
396  VAL  THR  10.00  10.00  10.00  10.00  10.00   5.84   6.15  -1.56  ...   
397  VAL  TRP  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.27  ...   
398  VAL  TYR  10.00  10.00  10.00  10.00  10.00  10.00   5.71  -1.38  ...   
399  VAL  VAL  10.00  10.00  10.00  10.00  10.00  10.00  10.00  -1.48  ...   

     10.25  10.75  11.25  11.75  12.25  12.75  13.25  13.75  14

# Recherche de la distance dans la DOPE depuis les deux résidus

In [4]:
AA_CODES = {
    'A': 'ALA', 'R': 'ARG', 'N': 'ASN', 'D': 'ASP', 'C': 'CYS',
    'Q': 'GLN', 'E': 'GLU', 'G': 'GLY', 'H': 'HIS', 'I': 'ILE',
    'L': 'LEU', 'K': 'LYS', 'M': 'MET', 'F': 'PHE', 'P': 'PRO',
    'S': 'SER', 'T': 'THR', 'W': 'TRP', 'Y': 'TYR', 'V': 'VAL'
}

In [5]:
def round_distance(dist):
    # arrondi au quart le plus proche
    rounded_value = round(dist * 4) / 4
    
    # ne garde que 0.25 ou 0.75
    decimal = rounded_value % 1
    if decimal == 0.0:
        return rounded_value + 0.25
    elif decimal == 0.5:
        return rounded_value + 0.25
    else:
        return rounded_value
    # si pile au milieu, on prends le plus proche supérieur

dist = 10.5
round_distance(dist)

10.75

In [16]:
def get_potential(res1, res2, dist, dope):
    closest_dist = round_distance(dist)
    potential = dope.loc[(dope['res1'] == res1) & (dope['res2'] == res2), closest_dist]
    print(float(potential.iloc[0]))

In [17]:
get_potential(AA_CODES['A'], AA_CODES['R'], 10.2, DOPE_MATRIX)

-0.13


# Construction des low level

In [ ]:
# 5awl
seq = "YYDPETGTWY"
longueur = 10 # template.longueur

for aa in enumerate(seq):
    for position in range(longueur):
        # code

        

In [ ]:
class DynamicMatrix:
    def __init__(self, lines, columns, score_matrix, gap):
        self.matrix = np.zeros((lines, columns))
        self.lines = lines
        self.columns = columns
        self.score_matrix = score.matrix
        self.gap = gap

    def fill_matrix(self, get_score):
        for i in range(1, lines):
            for j in range(1, columns):
                get_score()

In [64]:
class LowLevelMatrix:
    def __init__(self, lines, columns, score_matrix, gap, fixed_aa_id, fixed_pos):
        self.matrix = np.zeros((lines, columns))
        self.lines = lines
        self.columns = columns
        self.score_matrix = score_matrix
        self.gap = gap

        # Vérification du blocage de la case
        if (fixed_aa_id >= lines) or (fixed_aa_id < 0):
            raise ValueError("Ligne fixée hors de la matrice")
        if (fixed_pos >= columns) or (fixed_pos < 0):
            raise ValueError("Colonne fixée hors de la matrice")
        
        self.fixed_aa_id = fixed_aa_id
        self.fixed_pos = fixed_pos

    def initialize_matrix(self, first_val, start, end):
        # Première case
        self.matrix[start[0], start[1]] = first_val
        
        # Remplissage de la première colonne jusqu'à la limite
        for i in range(start[0] + 1, end[0] + 1):
            self.matrix[i, start[1]] = self.matrix[i - 1, start[1]] + self.gap

        # Remplissage de la première ligne jusqu'à la limite
        for j in range(start[1] + 1, end[1] + 1):
            self.matrix[start[0], j] = self.matrix[start[0], j - 1] + self.gap

    def fill_matrix(self):
        # MANQUE LES SORTIES DE MATRICE
        
        # Partie supérieure gauche
        self.initialize_matrix(0, [0, 0], [self.fixed_aa_id - 1, self.fixed_pos - 1])

        for i in range(1, self.fixed_aa_id):
            for j in range(1, self.fixed_pos):
                self.matrix[i, j] = min(self.matrix[i - 1, j - 1] + SCORE, self.matrix[i - 1] + self.gap, self.matrix[i, j - 1] + self.gap)

        # Case fixée
        self.matrix[self.fixed_aa_id, self.fixed_pos] = self.matrix[self.fixed_aa_id - 1, self.fixed_pos - 1]
        
        # Partie inférieure droite
        self.initialize_matrix(self.matrix[self.fixed_aa_id, self.fixed_pos],
                               [self.fixed_aa_id + 1, self.fixed_pos + 1],
                               [self.lines, self.columns]
                              )

        for i in range(self.fixed_aa_id + 2, self.lines):
            for j in range(self.fixed_pos + 2, self.columns):
                self.matrix[i, j] = min(self.matrix[i - 1, j - 1] + SCORE, self.matrix[i - 1] + self.gap, self.matrix[i, j - 1] + self.gap)

In [65]:
m = LowLevelMatrix(5, 5, "rien", -2, 2, 2)
print(m.matrix, "\n")
m.initialize_matrix(8, [0, 0], [m.fixed_aa_id - 1, m.fixed_pos - 1])
print(m.matrix, "\n")
m.matrix[m.fixed_aa_id, m.fixed_pos] = 5
print(m.matrix, "\n")
m.initialize_matrix(m.matrix[m.fixed_aa_id, m.fixed_pos],
                               [m.fixed_aa_id + 1, m.fixed_pos + 1],
                               [m.lines - 1, m.columns - 1]
                              )
print(m.matrix, "\n")

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]] 

[[8. 6. 0. 0. 0.]
 [6. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]] 

[[8. 6. 0. 0. 0.]
 [6. 0. 0. 0. 0.]
 [0. 0. 5. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]] 

[[8. 6. 0. 0. 0.]
 [6. 0. 0. 0. 0.]
 [0. 0. 5. 0. 0.]
 [0. 0. 0. 5. 3.]
 [0. 0. 0. 3. 0.]] 



In [66]:
m = LowLevelMatrix(5, 5, "rien", -2, 5, 2)

ValueError: Ligne fixée hors de la matrice

In [67]:
m = LowLevelMatrix(5, 5, "rien", -2, 2, -2)

ValueError: Colonne fixée hors de la matrice

# Classes matrices dynamiques

In [33]:
class DynamicMatrix:
    def __init__(self, lines, columns, score_matrix, gap):
        self.matrix = np.zeros((lines, columns))
        self.lines = lines
        self.columns = columns
        self.score_matrix = score_matrix
        self.gap = gap

    def initialize_matrix(self, first_val, start, end):
        if (start[0] < 0) or (start[1] < 0):
            raise ValueError("Start of initialization out of matrix.")
        if (end[0] >= self.lines) or (end[1] >= self.columns):
            raise ValueError("End of initialization out of matrix.")
        
        # Première case
        self.matrix[start[0], start[1]] = first_val
        
        # Remplissage de la première colonne jusqu'à la limite
        for i in range(start[0] + 1, end[0] + 1):
            self.matrix[i, start[1]] = self.matrix[i - 1, start[1]] + self.gap

        # Remplissage de la première ligne jusqu'à la limite
        for j in range(start[1] + 1, end[1] + 1):
            self.matrix[start[0], j] = self.matrix[start[0], j - 1] + self.gap

In [49]:
class LowLevelMatrix(DynamicMatrix):
    def __init__(self, lines, columns, score_matrix, gap, fixed_aa_id, fixed_pos):
        DynamicMatrix.__init__(self, lines, columns, score_matrix, gap)

        # Vérification du blocage de la case
        if (fixed_aa_id >= lines) or (fixed_aa_id < 0):
            raise ValueError("Frozen line index out of matrix.")
        if (fixed_pos >= columns) or (fixed_pos < 0):
            raise ValueError("Frozen column index out of matrix")
        
        self.fixed_aa_id = fixed_aa_id
        self.fixed_pos = fixed_pos

    def fill_matrix(self):
        # Partie supérieure gauche
        self.initialize_matrix(0, [0, 0], [self.fixed_aa_id - 1, self.fixed_pos - 1])

        for i in range(1, self.fixed_aa_id):
            for j in range(1, self.fixed_pos):
                self.matrix[i, j] = min(self.matrix[i - 1, j - 1] + SCORE,
                                        self.matrix[i - 1] + self.gap,
                                        self.matrix[i, j - 1] + self.gap
                                       )

        # Case fixée
        self.matrix[self.fixed_aa_id, self.fixed_pos] = self.matrix[self.fixed_aa_id - 1, self.fixed_pos - 1]
        
        # Partie inférieure droite
        self.initialize_matrix(self.matrix[self.fixed_aa_id, self.fixed_pos],
                               [self.fixed_aa_id + 1, self.fixed_pos + 1],
                               [self.lines, self.columns]
                              )

        for i in range(self.fixed_aa_id + 2, self.lines):
            for j in range(self.fixed_pos + 2, self.columns):
                self.matrix[i, j] = min(self.matrix[i - 1, j - 1] + SCORE,
                                        self.matrix[i - 1] + self.gap,
                                        self.matrix[i, j - 1] + self.gap
                                       )

In [35]:
dyn_test = DynamicMatrix(5, 5, "rien", -1)
dyn_test.initialize_matrix(8, [0, 0], [4, 4])
print(dyn_test.matrix)

[[8. 7. 6. 5. 4.]
 [7. 0. 0. 0. 0.]
 [6. 0. 0. 0. 0.]
 [5. 0. 0. 0. 0.]
 [4. 0. 0. 0. 0.]]


In [48]:
dyn_test = LowLevelMatrix(5, 5, "rien", -1, 0, 4)
dyn_test.initialize_matrix(8, [0, 0], [dyn_test.fixed_aa_id - 1, dyn_test.fixed_pos - 1])
print(dyn_test.matrix)

[[8. 7. 6. 5. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
